In [1]:
# setting up the environment
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, QuantileTransformer
from env import user, password, host
import env
from sklearn.feature_selection import SelectKBest, f_regression, RFE
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import csv
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
# create a function for the acquisition 
def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
# acquire required tables into one dataframe
def acquire_zillow():
    file='zillow_df.csv'
    if os.path.isfile(file):
        return pd.read_csv(file)
    else:
        zillow2017_df = pd.read_sql(('''SELECT
    prop.*,
    predictions_2017.logerror,
    predictions_2017.transactiondate,
    air.airconditioningdesc,
    arch.architecturalstyledesc,
    build.buildingclassdesc,
    heat.heatingorsystemdesc,
    landuse.propertylandusedesc,
    story.storydesc,
    construct.typeconstructiondesc
FROM properties_2017 prop
JOIN (
    SELECT parcelid, MAX(transactiondate) max_transactiondate
    FROM predictions_2017
    GROUP BY parcelid
) pred USING(parcelid)
JOIN predictions_2017 ON pred.parcelid = predictions_2017.parcelid
                      AND pred.max_transactiondate = predictions_2017.transactiondate
LEFT JOIN airconditioningtype air USING (airconditioningtypeid)
LEFT JOIN architecturalstyletype arch USING (architecturalstyletypeid)
LEFT JOIN buildingclasstype build USING (buildingclasstypeid)
LEFT JOIN heatingorsystemtype heat USING (heatingorsystemtypeid)
LEFT JOIN propertylandusetype landuse USING (propertylandusetypeid)
LEFT JOIN storytype story USING (storytypeid)
LEFT JOIN typeconstructiontype construct USING (typeconstructiontypeid)
WHERE prop.latitude IS NOT NULL
  AND prop.longitude IS NOT NULL
  AND predictions_2017.transactiondate like "2017%%" '''), get_connection('zillow'))
        zillow2017_df.to_csv(file,index=False)
    return zillow2017_df


In [4]:
def zillow_info():
    zillow_df=acquire_zillow()
    print('The basic stats about zillow dataframe is: ','\n',zillow_df.describe())
    print('The shape of the zillow dataframe is: ','\n',zillow_df.info())

In [5]:
zillow_info()

The basic stats about zillow dataframe is:  
                  id      parcelid  airconditioningtypeid  \
count  7.738000e+04  7.738000e+04           24953.000000   
mean   1.495126e+06  1.300715e+07               1.813289   
std    8.609057e+05  3.481368e+06               2.967894   
min    3.490000e+02  1.071186e+07               1.000000   
25%    7.520500e+05  1.153830e+07               1.000000   
50%    1.497870e+06  1.253155e+07               1.000000   
75%    2.240480e+06  1.421184e+07               1.000000   
max    2.982274e+06  1.676893e+08              13.000000   

       architecturalstyletypeid  basementsqft   bathroomcnt    bedroomcnt  \
count                206.000000     50.000000  77380.000000  77380.000000   
mean                   7.388350    679.720000      2.299134      3.053489   
std                    2.734542    689.703546      0.996657      1.139103   
min                    2.000000     38.000000      0.000000      0.000000   
25%                    7.000

## Key takeaway
There are 67 columns, a lot of columns are missing over 70000 values.
There are 66 float, 2 int and 11 object
The total count is 77375 rows retrieved

In [6]:
zillow_df=acquire_zillow()
sumdf=pd.DataFrame(zillow_df.isnull().sum(axis=0).rename('total'))
meandf=pd.DataFrame(zillow_df.isnull().mean(axis=0).rename('percentage'))
pd.concat([sumdf,meandf],axis=1)

,total,percentage
id,0,0.000000
parcelid,0,0.000000
airconditioningtypeid,52427,0.677526
architecturalstyletypeid,77174,0.997338
basementsqft,77330,0.999354
...,...,...
buildingclassdesc,77365,0.999806
heatingorsystemdesc,27941,0.361088
propertylandusedesc,0,0.000000
storydesc,77330,0.999354


## Takeaway
- A lot of columns have missing over 90% of the values. the problem is that whether we should decided to drop the null values
- Some of missing values can be dropped, which like ids. However, some values we can imputed

In [7]:
# function for null counts 
def null_counts():
    sumdf=pd.DataFrame(zillow_df.isnull().sum(axis=0).rename('num_rows_missing'))
    meandf=pd.DataFrame(zillow_df.isnull().mean(axis=0).rename('pct_rows_missing'))
    print(pd.concat([sumdf,meandf],axis=1))
    

In [8]:
# clean the column and create county columns
def clean_column():
    zillow2017_df=acquire_zillow()
    zillow2017_df.rename(columns={'bedroomcnt':'bedroom','bathroomcnt':'bathroom','calculatedfinishedsquarefeet':'sqtft','taxvaluedollarcnt':'taxvalue','garagecarcnt':'garage','lotsizesquarefeet':'lots','poolcnt':'pool','regionidzip':'zipcode'},inplace=True)
    zillow2017_df['fips']= zillow2017_df['fips'].astype(object)
    value=[]
    for row in zillow2017_df['fips']:
        if row ==6037.0: value.append('Los Angeles County, CA')
        elif row == 6059.0: value.append("Orange County, CA")
        elif row == 6111.0: value.append('Ventura County, CA')
        else:
            value.append('Undetermined')
    zillow2017_df['county']=value
    zillow2017_df['zipcode']=zillow2017_df['zipcode'].astype(object)
    zillow2017_df= zillow2017_df[zillow2017_df.columns.drop(list(zillow2017_df.filter(regex='id')))]
    zillow2017_df['taxrate']=zillow2017_df.taxamount/zillow2017_df.taxvalue
    return zillow2017_df

In [9]:
# create function that will drop null when exceed certain amount of null values
def handle_missing_values(df, prop_required_column, prop_required_row):
    n_required_column = round(df.shape[0] * prop_required_column)
    n_required_row = round(df.shape[1] * prop_required_row)
    df = df.dropna(axis=0, thresh=n_required_row)
    df = df.dropna(axis=1, thresh=n_required_column)
    return df

In [10]:
def wrangle_zillow():
    zillow_df=clean_column()
    zillow2017_df=handle_missing_values(zillow_df,.4,.5)
    train_validate, zillow_test = train_test_split(zillow2017_df, test_size=.2, random_state=123)
    zillow_train, zillow_validate = train_test_split(train_validate, test_size=.3, random_state=123)
    zillow_train['month']=pd.DatetimeIndex(zillow_train['transactiondate']).month
    zillow_train=zillow_train.drop(columns=['transactiondate','heatingorsystemdesc','unitcnt','propertyzoningdesc','lots'])
    zillow_validate['month']=pd.DatetimeIndex(zillow_validate['transactiondate']).month
    zillow_validate=zillow_validate.drop(columns=['transactiondate','heatingorsystemdesc','unitcnt','propertyzoningdesc','lots'])
    zillow_test['month']=pd.DatetimeIndex(zillow_test['transactiondate']).month
    zillow_test=zillow_test.drop(columns=['transactiondate','heatingorsystemdesc','unitcnt','propertyzoningdesc','lots'])
    return zillow_train, zillow_validate, zillow_test

In [11]:
zillow_train, zillow_validate, zillow_test=wrangle_zillow()

In [12]:
zillow_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42993 entries, 39641 to 7566
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   bathroom                    42993 non-null  float64
 1   bedroom                     42993 non-null  float64
 2   calculatedbathnbr           42816 non-null  float64
 3   sqtft                       42991 non-null  float64
 4   finishedsquarefeet12        41116 non-null  float64
 5   fips                        42993 non-null  object 
 6   fullbathcnt                 42816 non-null  float64
 7   latitude                    42993 non-null  float64
 8   longitude                   42993 non-null  float64
 9   propertycountylandusecode   42993 non-null  object 
 10  rawcensustractandblock      42993 non-null  float64
 11  zipcode                     42971 non-null  object 
 12  roomcnt                     42993 non-null  float64
 13  yearbuilt                   

In [13]:
zillow_train.isnull().sum()

bathroom                         0
bedroom                          0
calculatedbathnbr              177
sqtft                            2
finishedsquarefeet12          1877
fips                             0
fullbathcnt                    177
latitude                         0
longitude                        0
propertycountylandusecode        0
rawcensustractandblock           0
zipcode                         22
roomcnt                          0
yearbuilt                       32
structuretaxvaluedollarcnt      52
taxvalue                         1
assessmentyear                   0
landtaxvaluedollarcnt            1
taxamount                        3
censustractandblock            124
logerror                         0
propertylandusedesc              0
county                           0
taxrate                          4
month                            0
dtype: int64

In [14]:
zillow_train.isnull().sum().max()
imp = SimpleImputer(strategy="most_frequent")

In [15]:
from datetime import datetime
def wrangled_file():
    zillow_train,zillow_validate,zillow_test=wrangle_zillow()
    zillow_train=zillow_train.dropna(axis=0)
    zillow_validate=zillow_validate.dropna(axis=0)
    zillow_test=zillow_test.dropna(axis=0)
    zillow_train['age']=2022.0-zillow_train.yearbuilt
    zillow_validate['age']=2022.0-zillow_validate.yearbuilt
    zillow_test['age']=2022.0-zillow_test.yearbuilt
    return zillow_train,zillow_validate,zillow_test

In [16]:
zillow_train,zv,zt=wrangled_file()
zillow_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40900 entries, 39641 to 7566
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   bathroom                    40900 non-null  float64
 1   bedroom                     40900 non-null  float64
 2   calculatedbathnbr           40900 non-null  float64
 3   sqtft                       40900 non-null  float64
 4   finishedsquarefeet12        40900 non-null  float64
 5   fips                        40900 non-null  object 
 6   fullbathcnt                 40900 non-null  float64
 7   latitude                    40900 non-null  float64
 8   longitude                   40900 non-null  float64
 9   propertycountylandusecode   40900 non-null  object 
 10  rawcensustractandblock      40900 non-null  float64
 11  zipcode                     40900 non-null  object 
 12  roomcnt                     40900 non-null  float64
 13  yearbuilt                   

NameError: name 'imputer' is not defined

In [ ]:
zillow_train.select_dtypes(float).columns

In [ ]:
    zillow_validate=pd.DataFrame(zillow_validate,columns=[calculatedbathnbr])
    zillow_test=pd.DataFrame(zillow_test,columns=zillow_col)

In [ ]:
illow_train['taxrate','heatingorsystemdesc','censustractandblock','structuretaxvaluedollarcnt']=pd.DataFrame(imputer.transform(zillow_train['taxrate','heatingorsystemdesc','censustractandblock','structuretaxvaluedollarcnt']))
    zillow_train['age']=2022.0-zillow_train.yearbuilt.astype(float)